# Exploring OpenAI V1 functionality

On 11.06.23 OpenAI released a number of new features, and along with it bumped their Python SDK to 1.0.0. This notebook shows off the new features and how to use them with LangChain.

In [ ]:
!pip install -U openai "langchain>=0.0.331rc1" langchain-experimental

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

## [Vision](https://platform.openai.com/docs/guides/vision)

OpenAI released multi-modal models, which can take a sequence of text and images as input.

In [26]:
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)
chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "What is this image showing"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/static/img/langchain_stack.png",
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)

AIMessage(content='This image appears to be a diagram representing the architecture or components of a software system or platform named "LangChain." The diagram is organized into various blocks that represent different layers or aspects of the system. Here\'s a breakdown of the labeled parts:\n\n1. **LangSmith**: This seems to be a tool or component related to testing, evaluation, monitoring, feedback, and annotation, as well as debugging.\n\n2. **LangServe**: This component chains as REST API, suggesting it may serve as an interface for the system\'s functionality to be accessed over the web using RESTful API calls.\n\n3. **Templates**: Reference applications are mentioned here, which might be pre-built applications or use cases provided as starting points for users of the system.\n\n4. **Chains, agents, agent executors**: This part refers to common application logic, indicating modular components that can be combined or chained together to create complex workflows.\n\n5. **Model I/O

## [OpenAI assistants](https://platform.openai.com/docs/assistants/overview)

> The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling


You can interact with OpenAI Assistants using OpenAI tools or custom tools. When using exclusively OpenAI tools, you can just invoke the assistant directly and get final answers. When using custom tools, you can run the assistant and tool execution loop using the built-in AgentExecutor or easily write your own executor.

Below we show the different ways to interact with Assistants. As a simple example, let's build a math tutor that can write and run code.

### Using only OpenAI tools

In [1]:
from langchain_experimental.openai_assistant import OpenAIAssistantRunnable

In [3]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

[ThreadMessage(id='msg_ZW0b5sZZbTSr5vZdpRCAsFgx', assistant_id='asst_7Sjc4Alo4710XHfTIxlksqY3', content=[MessageContentText(text=Text(annotations=[], value='The result of the expression \\( 10 - 4^{2.7} \\) is approximately -32.224.'), type='text')], created_at=1699409928, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_ytM7dqRyb0ic3RQQ5R9dvMu5', thread_id='thread_ihggZkCBGQn9y0igas3HRJAC')]

### As a LangChain agent with arbitrary tools

Now let's recreate this functionality using our own tools. For this example we'll use the [E2B sandbox runtime tool](https://e2b.dev/docs?ref=landing-page-get-started).

In [2]:
from langchain.tools import E2BDataAnalysisTool

tools = [E2BDataAnalysisTool(api_key="e2b_c4547586ee874bb331944cfbf8cdf12fd64cfb59")]

In [3]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

#### Using AgentExecutor

In [6]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's 10 - 4 raised to the 2.7"})

ValueError: Unknown run status failed. Full run info:

{
  "id": "run_7l5Nm8EqHrfFDfixhDvROzJH",
  "assistant_id": "asst_wIA9zMaoHpER3clGiCmb5fnF",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1699411059,
  "expires_at": null,
  "failed_at": 1699411073,
  "file_ids": [],
  "instructions": "You are a personal math tutor. Write and run code to answer math questions.",
  "last_error": {
    "code": "rate_limit_exceeded",
    "message": "We're currently processing too many requests - please try again later."
  },
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": null,
  "started_at": 1699411063,
  "status": "failed",
  "thread_id": "thread_qbv6746Cf2RFK58Cykb4kpm6",
  "tools": [
    {
      "function": {
        "description": "Evaluates python code in a sandbox environment. The environment is long running and exists across multiple executions. You must send the whole script every time and print your outputs. Script should 

#### Custom execution

In [ ]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

In [ ]:
from langchain.schema.agent import AgentFinish

def execute_agent(agent, tools, input):
    tool_map = {tool.name: tool for tool in tools}
    response = agent.invoke(input)
    while not isinstance(response, AgentFinish):
        tool_outputs = []
        for action in response:
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            tool_outputs.append({"output": tool_output, "tool_call_id": action.tool_call_id})
        response = agent.invoke({"tool_outputs": tool_outputs, "run_id": action.run_id, "thread_id": action.thread_id})
        
    return response

In [ ]:
response = execute_agent(agent, tools, {"content": "What's 10 - 4 raised to the 2.7"})
print(response.return_values["output"])

In [ ]:
next_response = execute_agent(agent, tools, {"content": "now add 17.241", "thread_id": response.thread_id})
print(next_response.return_values["output"])

## [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode)

Constrain the model to only generate valid JSON. Note that you must include a system message with instructions to use JSON for this mode to work.

Only works with certain models. 

In [16]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    response_format={"type": "json_object"}
)

output = chat.invoke(
    [
        SystemMessage(
            content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
        ),
        HumanMessage(
            content="Google was founded in the USA, while Deepmind was founded in the UK"
        ),
    ]
)
print(output.content)

{
  "companies": [
    {
      "name": "Google",
      "origin": "USA"
    },
    {
      "name": "Deepmind",
      "origin": "UK"
    }
  ]
}


In [17]:
import json

json.loads(output.content)

{'companies': [{'name': 'Google', 'origin': 'USA'},
  {'name': 'Deepmind', 'origin': 'UK'}]}

## [System fingerprint](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

OpenAI sometimes changes model configurations in a way that impacts outputs. Whenever this happens, the system_fingerprint associated with a generation will change.

In [18]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
output = chat.generate(
    [
        [
            SystemMessage(
                content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
            ),
            HumanMessage(
                content="Google was founded in the USA, while Deepmind was founded in the UK"
            ),
        ]
    ]
)
print(output.llm_output)

{'token_usage': {'completion_tokens': 43, 'prompt_tokens': 49, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_eeff13170a'}
